# Worksheet 08

Name:  Albert Cheng
UID:    U55565220

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [5]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.059326299592268, 5.02534380024279, 6.018055191199706, 2.608328925559854, 5.420056833104754, 5.139328895917362, 6.493770350783252, 4.823107116040532, 5.712490326996658, 4.733906228678958]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [6]:
mean = 8
stdev = 1
c2 = np.random.normal(mean, stdev, 10 ).tolist()
print(c2)

[4.974710152469152, 6.586628796248338, 7.492346057509461, 9.041732770765602, 8.453079176640859, 9.604971852922361, 8.935684151665244, 8.155484834035356, 8.28315227686027, 7.057714648846334]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [7]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[7.057714648846334, 8.28315227686027, 8.155484834035356, 4.733906228678958, 8.935684151665244, 5.712490326996658, 9.604971852922361, 8.453079176640859, 4.823107116040532, 9.041732770765602]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

A mean
A covariance 
mixing probability


d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [12]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [len(c1) / (len(c1) + len(c2)), len(c2) / (len(c1) + len(c2))]
mean = [ sum(c1)/len(c1) , sum(c2) / len(c2)]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum((x - mean[1]) ** 2 for x in c2) / len(c2)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.057714648846334, 8.28315227686027, 8.155484834035356, 8.935684151665244, 9.604971852922361, 8.453079176640859, 9.041732770765602]
[4.733906228678958, 5.712490326996658, 4.823107116040532]
P(C_1) = 0.7,  P(C_2) = 0.3
mean_1 = 8.504545673105147,  mean_2 = 5.089834557238716
var_1 = 0.5646022087291642,  var_2 = 0.1951762368574431


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [13]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0]  / prob_x)
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  7.057714648846334
probability of observing that point if it came from cluster 0 =  0.026498168531707514
probability of observing that point if it came from cluster 1 =  1.7204270579290856e-22
point =  8.28315227686027
probability of observing that point if it came from cluster 0 =  0.6543029389928298
probability of observing that point if it came from cluster 1 =  1.5225607065397141e-58
point =  8.155484834035356
probability of observing that point if it came from cluster 0 =  0.5836718752437983
probability of observing that point if it came from cluster 1 =  5.464137164711425e-54
point =  4.733906228678958
probability of observing that point if it came from cluster 0 =  1.4594101296438646e-10
probability of observing that point if it came from cluster 1 =  0.38755757982508815
point =  8.935684151665244
probability of observing that point if it came from cluster 0 =  0.5278947799388038
probability of observing that point if it came from cluster 1 =  9.984597044017186e-85
point

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [14]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x) / len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [sum([x * (data[i] - mean[0]) ** 2 for i, x in enumerate(prob_c0_x)]) / sum(prob_c0_x),
       sum([x * (data[i] - mean[1]) ** 2 for i, x in enumerate(prob_c1_x)]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.7000639924028815,  P(C_2) = 0.29993600759711836
mean_1 = 8.504290452831814,  mean_2 = 5.0897017120819195
var_1 = 0.5652631238181602,  var_2 = 0.1951351437461945


The values are very similar. 

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [16]:
updated_c0_x = []  
updated_c1_x = []  

for p in data:
    # Calculate P(X_i | C_j)
    pdf_c0 = norm.pdf(p, mean[0], var[0])
    pdf_c1 = norm.pdf(p, mean[1], var[1])

    # Calculate updated P(C_j | X_i) using Bayes' theorem
    updated_c0 = (pdf_c0 * prob_c[0]) / (pdf_c0 * prob_c[0] + pdf_c1 * prob_c[1])
    updated_c1 = (pdf_c1 * prob_c[1]) / (pdf_c0 * prob_c[0] + pdf_c1 * prob_c[1])

    updated_c0_x.append(updated_c0)
    updated_c1_x.append(updated_c1)
for i in range(len(data)):
    print(f"Point: {data[i]}")
    print(f"Previous P(C_0 | X_i): {prob_c0_x[i]}, Updated P(C_0 | X_i): {updated_c0_x[i]}")
    print(f"Previous P(C_1 | X_i): {prob_c1_x[i]}, Updated P(C_1 | X_i): {updated_c1_x[i]}\n")

Data Point: 7.057714648846334
Previous P(C_0 | X_i): 1.0, Updated P(C_0 | X_i): 1.0
Previous P(C_1 | X_i): 2.7825541266647522e-21, Updated P(C_1 | X_i): 2.6840850491202467e-21

Data Point: 8.28315227686027
Previous P(C_0 | X_i): 1.0, Updated P(C_0 | X_i): 1.0
Previous P(C_1 | X_i): 9.972842519901928e-59, Updated P(C_1 | X_i): 9.328422484933149e-59

Data Point: 8.155484834035356
Previous P(C_0 | X_i): 1.0, Updated P(C_0 | X_i): 1.0
Previous P(C_1 | X_i): 4.012139645434274e-54, Updated P(C_1 | X_i): 3.769789755783551e-54

Data Point: 4.733906228678958
Previous P(C_0 | X_i): 8.78654031498274e-10, Updated P(C_0 | X_i): 9.26953944586357e-10
Previous P(C_1 | X_i): 0.999999999121346, Updated P(C_1 | X_i): 0.999999999073046

Data Point: 8.935684151665244
Previous P(C_0 | X_i): 1.0, Updated P(C_0 | X_i): 1.0
Previous P(C_1 | X_i): 8.105996083841868e-85, Updated P(C_1 | X_i): 7.375415692739903e-85

Data Point: 5.712490326996658
Previous P(C_0 | X_i): 0.000639921947835198, Updated P(C_0 | X_i): 0

The values are very similar

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      agree       |
| A  C |      agree       |
| A  D |      disagree       |
| A  E |      agree       |
| B  C |      agree       |
| B  D |      agree       |
| B  E |      disagree       |
| C  D |      disagree       |
| C  E |      agree       |
| D  E |      agree       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

I dont know

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

I got no clue

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

Nothing